In [1]:
import numpy as np
import tensorflow as tf
import wave, math, struct
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation

Prepare a dummy data for simulating notes

In [2]:
notes_freq={'A':440.0,'B':493.88,'C':261.63,'D':293.66,'E':393.63,'F':349.23,'G':392.00}
notes_freq

{'A': 440.0,
 'B': 493.88,
 'C': 261.63,
 'D': 293.66,
 'E': 393.63,
 'F': 349.23,
 'G': 392.0}

In [3]:
notes=list(notes_freq.keys())
notes

['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [4]:
note_to_int={note:i for i,note in enumerate(notes)}
note_to_int

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}

In [5]:
int_to_note={i:note for i,note in enumerate(notes)}
int_to_note

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G'}

In [6]:
raw_music_data=[notes[np.random.randint(0,7)]for i in range(1000)]
raw_music_data

['B',
 'B',
 'F',
 'E',
 'D',
 'B',
 'G',
 'F',
 'B',
 'G',
 'B',
 'B',
 'C',
 'E',
 'C',
 'B',
 'C',
 'A',
 'B',
 'A',
 'E',
 'F',
 'C',
 'F',
 'D',
 'B',
 'B',
 'B',
 'B',
 'G',
 'A',
 'D',
 'E',
 'B',
 'F',
 'D',
 'E',
 'D',
 'B',
 'D',
 'C',
 'B',
 'C',
 'B',
 'G',
 'B',
 'D',
 'F',
 'D',
 'F',
 'G',
 'D',
 'A',
 'B',
 'C',
 'D',
 'F',
 'D',
 'A',
 'D',
 'C',
 'C',
 'F',
 'F',
 'C',
 'D',
 'A',
 'D',
 'G',
 'D',
 'E',
 'C',
 'G',
 'C',
 'B',
 'A',
 'F',
 'G',
 'A',
 'B',
 'A',
 'F',
 'E',
 'G',
 'D',
 'C',
 'C',
 'B',
 'D',
 'A',
 'A',
 'F',
 'G',
 'B',
 'G',
 'B',
 'D',
 'E',
 'B',
 'G',
 'D',
 'A',
 'C',
 'E',
 'A',
 'G',
 'A',
 'G',
 'B',
 'E',
 'C',
 'B',
 'F',
 'G',
 'G',
 'F',
 'G',
 'A',
 'F',
 'E',
 'D',
 'D',
 'D',
 'B',
 'C',
 'F',
 'D',
 'F',
 'A',
 'G',
 'D',
 'G',
 'D',
 'E',
 'B',
 'D',
 'D',
 'E',
 'C',
 'E',
 'D',
 'A',
 'A',
 'E',
 'C',
 'F',
 'G',
 'F',
 'B',
 'A',
 'F',
 'A',
 'E',
 'F',
 'B',
 'G',
 'D',
 'A',
 'D',
 'D',
 'C',
 'B',
 'A',
 'D',
 'G',
 'E',
 'E'

Data Preparation

In [7]:
sequence_length=3
network_input=[]
network_output=[]

for i in range(len(raw_music_data)-sequence_length):
    seq_in=raw_music_data[i:i+sequence_length]
    seq_out=raw_music_data[i+sequence_length]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])
    print(seq_in,'-->',seq_out)

['B', 'B', 'F'] --> E
['B', 'F', 'E'] --> D
['F', 'E', 'D'] --> B
['E', 'D', 'B'] --> G
['D', 'B', 'G'] --> F
['B', 'G', 'F'] --> B
['G', 'F', 'B'] --> G
['F', 'B', 'G'] --> B
['B', 'G', 'B'] --> B
['G', 'B', 'B'] --> C
['B', 'B', 'C'] --> E
['B', 'C', 'E'] --> C
['C', 'E', 'C'] --> B
['E', 'C', 'B'] --> C
['C', 'B', 'C'] --> A
['B', 'C', 'A'] --> B
['C', 'A', 'B'] --> A
['A', 'B', 'A'] --> E
['B', 'A', 'E'] --> F
['A', 'E', 'F'] --> C
['E', 'F', 'C'] --> F
['F', 'C', 'F'] --> D
['C', 'F', 'D'] --> B
['F', 'D', 'B'] --> B
['D', 'B', 'B'] --> B
['B', 'B', 'B'] --> B
['B', 'B', 'B'] --> G
['B', 'B', 'G'] --> A
['B', 'G', 'A'] --> D
['G', 'A', 'D'] --> E
['A', 'D', 'E'] --> B
['D', 'E', 'B'] --> F
['E', 'B', 'F'] --> D
['B', 'F', 'D'] --> E
['F', 'D', 'E'] --> D
['D', 'E', 'D'] --> B
['E', 'D', 'B'] --> D
['D', 'B', 'D'] --> C
['B', 'D', 'C'] --> B
['D', 'C', 'B'] --> C
['C', 'B', 'C'] --> B
['B', 'C', 'B'] --> G
['C', 'B', 'G'] --> B
['B', 'G', 'B'] --> D
['G', 'B', 'D'] --> F
['B', 'D',

In [8]:
n_patterns=len(network_input)
n_patterns

997

In [9]:
X=np.reshape(network_input,(n_patterns,sequence_length,1))
X

array([[[1],
        [1],
        [5]],

       [[1],
        [5],
        [4]],

       [[5],
        [4],
        [3]],

       ...,

       [[2],
        [6],
        [2]],

       [[6],
        [2],
        [3]],

       [[2],
        [3],
        [0]]], shape=(997, 3, 1))

In [10]:
from keras.utils import to_categorical

In [11]:
y=to_categorical(network_output)

In [12]:
y.shape

(997, 7)

In [13]:
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], shape=(997, 7))

Build the model

In [14]:
model=Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2])))
model.add(Dense(1000))
model.add(Dense(7,activation='softmax'))

C:\Users\dai\PycharmProjects\PythonLab\Natural Language Processing\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

In [16]:
model.fit(X,y,epochs=100)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.1354 - loss: 1.9696
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1364 - loss: 1.9515
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1525 - loss: 1.9502
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1545 - loss: 1.9505
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1404 - loss: 1.9468
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1625 - loss: 1.9461
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1635 - loss: 1.9422
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1645 - loss: 1.9455
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1535 - loss: 1.9454
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1565 - loss: 1.9439
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1535 - loss: 1.9418
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc

Generate a new melody sequence

In [17]:
start_index=np.random.randint(0,len(network_input))
pattern=network_input[start_index]
pattern

[5, 6, 1]

In [18]:
generated_melody=[]
for i in range(16):
    x_input=np.reshape(pattern,(1,len(pattern),1))
    pred=model.predict(x_input,verbose=False)
    index=np.argmax(pred)
    result=int_to_note[index]
    generated_melody.append(result)
    pattern.append(index)
    pattern=pattern[1:len(pattern)]

In [19]:
pattern

[np.int64(4), np.int64(5), np.int64(3)]

In [20]:
generated_melody

['C',
 'F',
 'E',
 'D',
 'B',
 'B',
 'F',
 'E',
 'F',
 'D',
 'B',
 'B',
 'F',
 'E',
 'F',
 'D']

### Save as audio file

In [23]:
with wave.open('my_music.wav','w') as wav_file:
    #channel(mono), byte size,
    wav_file.setparams((1,2,44100,0,'NONE','not compressed'))
    for note in generated_melody:
        freq=notes_freq[note]
        num_samples=int(0.5*44100)#duration x sample rate
        for i in range(num_samples):
            #sample rate
            t=float(i)/44100
            value=int(32767*math.sin(2*math.pi*freq*t))
            data=struct.pack('<h',value)
            wav_file.writeframes(data)